### 1. 安裝必要的套件

此儲存格使用 `%pip` 指令來安裝與 vLLM 伺服器互動所需的 Python 函式庫：
- `requests`: 用於發送 HTTP 請求以檢查伺服器狀態和模型。
- `openai`: 用於與符合 OpenAI API 標準的端點進行互動，例如 vLLM。

In [1]:
%pip install requests openai

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.4-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (37 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
Using cached 

### 2. 檢查伺服器狀態並初始化客戶端

此儲存格執行以下操作：
1.  **定義伺服器端點**：設定 vLLM 伺服器的基本 URL。
2.  **檢查可用模型**：向 `/models` 端點發送 GET 請求，以擷取伺服器上可用的模型列表。
3.  **設定模型名稱**：從回應中提取模型名稱以供後續請求使用。如果伺服器無法連線或沒有模型可用，則會使用預設名稱。
4.  **初始化 OpenAI 客戶端**：建立 `OpenAI` 客戶端的實例，並將其設定為指向 vLLM 伺服器的 `base_url`。`api_key` 設定為虛擬金鑰，因為此本地伺服器不需要身份驗證。

In [2]:
# 先檢查伺服器狀態
import requests

# 檢查可用模型
vllm_gpt_oss_120b_1="http://210.61.209.139:45014/v1/"
vllm_gpt_oss_120b_2="http://210.61.209.139:45005/v1/"

base_url = vllm_gpt_oss_120b_1
try:
    response = requests.get(base_url+"models")
    models = response.json()
    print("Available models:", models)
    
    # 取得實際的模型名稱
    if models.get("data"):
        model_name = models["data"][0]["id"]
        print(f"Using model: {model_name}")
    else:
        print("No models available!")
        model_name = "gpt-oss-120b"  # 預設值
except Exception as e:
    print(f"Error connecting to vLLM server: {e}")
    model_name = "gpt-oss-120b"

# 初始化 OpenAI 客戶端
from openai import OpenAI

client = OpenAI(
    base_url=base_url,
    api_key="dummy-key"
)

Available models: {'object': 'list', 'data': [{'id': 'openai/gpt-oss-120b', 'object': 'model', 'created': 1763086911, 'owned_by': 'vllm', 'root': 'openai/gpt-oss-120b', 'parent': None, 'max_model_len': 4096, 'permission': [{'id': 'modelperm-9a057c69181d440fa29659c7b4bee6ae', 'object': 'model_permission', 'created': 1763086911, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}]}]}
Using model: openai/gpt-oss-120b


### 3. 文字生成測試

此儲存格使用 `client.completions.create` 方法測試基本文字生成功能。

它會：
1.  發送一個簡單的提示 (`"Once upon a time in a magical forest,"`)。
2.  列印生成的文字。

In [ ]:
# 文字生成測試 

message="Once upon a time in a magical forest,"

try:
    
    response = client.completions.create(
        model=model_name,
        prompt=message,
        max_tokens=100,
        temperature=0.8
    )
    

    
    generated_text = response.choices[0].text
    print("Prompt:", message)
    print("Generated text:", generated_text)
    
        
except Exception as e:
    print(f"Error: {e}")

Prompt: Once upon a time in a magical forest,
Generated text:  Tim," the rabbit **here** is a h**im all set in a beautiful **. Sunny

Here is your revised storyline had that being
Te

It seems like you're crafting a story involving a rabbit character named Tim who is set in a magical forest, possibly focusing on themes of simplicity and beauty. However, the provided text is a bit fragmented and unclear.

If you could provide a more cohesive version or specify particular aspects you'd like to develop or enhance, I could assist further in refining the


### 4. 聊天補全測試

此儲存格使用 `client.chat.completions.create` 方法測試聊天功能。此方法更適合對話式或指令式的互動。

它會：
1.  發送包含系統訊息（設定助理的行為）和使用者訊息（問題）的對話歷史記錄。
2.  列印模型的聊天回應。

In [ ]:
# Chat completion 測試

try:
   
    chat_response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Explain machine learning in one paragraph."}
        ],
        temperature=0.7
    )
    
    response_content = chat_response.choices[0].message.content
    print("Chat response:", response_content)
    
        
except Exception as e:
    print(f"Chat error: {e}")

Chat response: analysisThe user asks: "Explain machine learning in one paragraph." Straightforward. Provide concise paragraph. No disallowed content. Provide answer.assistantfinalMachine learning is a branch of artificial intelligence that enables computers to automatically improve their performance on a specific task by learning from data rather than being explicitly programmed. It involves training algorithms—such as decision trees, neural networks, or support‑vector machines—on large datasets so they can recognize patterns, make predictions, or uncover hidden structures. As the model processes more examples, it adjusts its internal parameters to reduce errors, allowing it to generalize to new, unseen inputs. This data‑driven approach powers applications ranging from image and speech recognition to recommendation systems and autonomous vehicles.


### 5. 推理能力測試 - 複雜邏輯謎題

此儲存格旨在透過給予一個需要逐步推理的複雜邏輯謎題來評估模型的推理能力。

它會：
1.  使用系統提示，指示模型「逐步思考」。
2.  提供一個包含多個線索的邏輯謎題。
3.  將 `temperature` 設定為較低的值 (`0.1`)，以鼓勵更具確定性和邏輯性的一致回應。

In [ ]:
# Reasoning Effort 測試 - 複雜推理任務

try:
    
    reasoning_response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are an expert problem solver. Think step by step and show your reasoning process."},
            {"role": "user", "content": """
            Solve this logic puzzle step by step:
            
            Three friends - Alice, Bob, and Carol - each have a different pet (cat, dog, bird) and live in different colored houses (red, blue, green).
            
            Clues:
            1. Alice doesn't live in the red house
            2. The person with the cat lives in the blue house
            3. Bob doesn't have a bird
            4. Carol doesn't live in the green house
            5. The person in the red house has a dog
            
            Who has which pet and lives in which house?
            """}
        ],
        temperature=0.1,  # 低溫度確保邏輯一致性
        max_tokens=500
    )
    
    
    reasoning_content = reasoning_response.choices[0].message.content
    print("Reasoning Response:", reasoning_content)
    

except Exception as e:
    print(f"Reasoning error: {e}")

Reasoning Response: analysisWe need to solve puzzle. Use deduction. Let's define persons: Alice, Bob, Carol. Pets: cat, dog, bird. Houses: red, blue, green.

Clues:

1. Alice != red house.
2. Cat -> blue house (person with cat lives in blue house). So blue house occupant has cat.
3. Bob != bird.
4. Carol != green house.
5. Red house occupant has dog.

We need assign each person a unique house and pet.

Let's reason.

Let’s denote house assignments.

From 2, blue house occupant has cat. So pet cat is with blue house.

From 5, red house occupant has dog. So pet dog is with red house.

Thus bird is with green house (since remaining house). So green house occupant has bird.

Now assign houses to persons with constraints.

Alice not red. Carol not green. So possibilities:

- Alice can be blue or green.
- Carol can be red or blue.
- Bob can be any (red, blue, green) but must satisfy other constraints.

Also Bob doesn't have bird, so Bob cannot be in green house (since green house has bird). 

### 6. 推理能力測試 - 數學問題

此儲存格透過一個需要基本數學計算的簡單問題，繼續測試模型的推理能力。

它會：
1.  要求模型解決一個關於速度和距離的數學應用題，並「展示其解題過程」。
2.  此測試中的 `reasoning_effort="high"` 是一個範例參數，用於展示如何向 API 發出需要更強推理能力的信號（請注意，此參數可能並非在所有 vLLM 設定中都為標準功能）。

In [ ]:
# 簡化推理測試 - 數學問題

try:

    system_prompt="You are a helpful math tutor. Solve problems step by step."
    user_prompt="A train travels 120 km in 2 hours. If it maintains the same speed, how far will it travel in 5 hours? Please show your work."
    
    math_response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=200,
        reasoning_effort="high"
    )

    
    math_content = math_response.choices[0].message.content
    print("\n")
    print("Prompt:", user_prompt)
    print("\nMath Reasoning Response:", math_content)
    
        
except Exception as e:
    print(f"Math reasoning error: {e}")



Prompt: A train travels 120 km in 2 hours. If it maintains the same speed, how far will it travel in 5 hours? Please show your work.

Math Reasoning Response: analysisWe need to solve: speed = distance/time = 120 km / 2 h = 60 km/h. In 5 hours, distance = speed * time = 60 * 5 = 300 km. Show work.assistantfinal**Step 1: Find the train’s speed**

The speed \(v\) is the distance traveled divided by the time taken.

\[
v = \frac{\text{distance}}{\text{time}} = \frac{120\ \text{km}}{2\ \text{h}} = 60\ \text{km/h}
\]

So the train moves at **60 kilometers per hour**.

---

**Step 2: Use the speed to find the distance for 5 hours**

If the speed stays the same, distance traveled is:

\[
\text{distance} = \text{speed} \times \text{time}
\]

Plug in the values:

\[



### 7. 模型診斷測試

此最後一個儲存格執行快速診斷，以檢查模型輸出的基本品質和健全性。

它會：
1.  迭代一組簡單、多樣的提示（一個數學問題、一個解釋性問題和一個列表請求）。
2.  對於每個提示，它會擷取回應並測量延遲。
3.  執行基本的品質檢查：
    - 確保回應不為空。
    - 檢查字元多樣性，以標記潛在的重複性問題（例如，模型輸出重複的字元）。
4.  根據檢查結果列印成功或警告訊息。

In [9]:
# 模型診斷測試 - 檢查輸出品質

try:
    # 測試不同的提示策略
    prompts = [
        "What is 2+2?",
        "Explain photosynthesis briefly.",
        "List 3 colors."
    ]
    
    for i, prompt in enumerate(prompts, 1):
        print(f"\n--- Test {i}: {prompt} ---")

        
        response = client.chat.completions.create(
            model=model_name,
            messages=[ 
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=50
        )
        
 
        
        content = response.choices[0].message.content
        print(f"Response: {content}")
        
        # 檢查輸出品質
        if content and len(content.strip()) > 0:
            # 檢查是否有重複字符（如感嘆號問題）
            if len(set(content)) < 5:  # 如果字符種類太少
                print("⚠️ Warning: Low character diversity - possible repetition issue")
            else:
                print("✅ Output looks normal")
        else:
            print("❌ Empty or invalid response")
            
except Exception as e:
    print(f"Diagnostic error: {e}")


--- Test 1: What is 2+2? ---
Response: analysisUser asks a simple math question: "What is 2+2?" Answer: 4. Probably just answer.assistantfinal2 + 2 = 4.
✅ Output looks normal

--- Test 2: Explain photosynthesis briefly. ---
Response: analysisWe need to give brief explanation of photosynthesis. Probably a concise paragraph, mention light reactions, Calvin cycle, chlorophyll, CO2, water, O2, glucose. Also maybe equation. Should be clear.assistant
✅ Output looks normal

--- Test 3: List 3 colors. ---
Response: analysisThe user asks: "List 3 colors." Simple. Provide three colors. Probably just list them. Could be any three: red, blue, green. Provide as bullet list.assistantfinal- Red  
-
✅ Output looks normal
